# NLP/LLM Engineer Test - Medical Fine-tuning

This notebook demonstrates the complete solution for the NLP/LLM Engineer test.
It covers:
1.  **Setup & Configuration**: Environment and hyperparameters.
2.  **Training**: Fine-tuning Qwen2.5-1.5B-Instruct on a medical dataset using LoRA.
3.  **Evaluation**: Comparing different LoRA configurations (r=8, 16, 32).
4.  **Inference**: Demonstrating the best model on diverse medical queries.


## 1. Setup & Dependencies


In [1]:
!pip install -r ../requirements.txt


Defaulting to user installation because normal site-packages is not writeable


In [2]:
import sys
import os
import yaml
import torch
from IPython.display import Markdown, display, Image
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# Add project root to path
sys.path.append(os.path.abspath("../"))
from src.utils.env_utils import setup_env

setup_env()
print("Environment setup complete.")


Setting HF_HOME to ./hf_cache
W&B API Key found.
Environment setup complete.


## 2. Configuration
We use `configs/config.yml` to manage all hyperparameters.


In [3]:
with open("../configs/config.yml", "r") as f:
    config = yaml.safe_load(f)
    print(yaml.dump(config, default_flow_style=False))


dataset:
  instruction_template: '<|im_start|>system

    You are a helpful medical assistant.<|im_end|>

    <|im_start|>user

    {question}<|im_end|>

    <|im_start|>assistant

    {answer}<|im_end|>

    '
  max_samples: 5000
  name: ruslanmv/ai-medical-chatbot
  seed: 42
  shuffle: true
  train_ratio: 0.9
  val_ratio: 0.1
evaluation:
  test_questions:
  - What are the symptoms of diabetes?
  - How to treat a common cold?
  - What causes high blood pressure?
  - What are the side effects of aspirin?
  - How to prevent heart disease?
  - What should I do if I have chest pain?
  - How can I lower my cholesterol naturally?
  - What are the early signs of cancer?
experiments:
- lora:
    lora_alpha: 32
    lora_dropout: 0.2
    r: 16
  name: lora_r16_alpha32
- dataset:
    max_samples: 1000
  lora:
    lora_alpha: 32
    lora_dropout: 0.2
    r: 16
  name: experiment_samples_1k
- dataset:
    max_samples: 5000
  lora:
    lora_alpha: 32
    lora_dropout: 0.2
    r: 16
  name: experime

## Task 2: Theory Questions

### 1. Comparison of Fine-tuning Methods
| Method | Description | Pros | Cons | When to use |
| :--- | :--- | :--- | :--- | :--- |
| **Full Fine-tuning** | Updates all model parameters. | Best potential performance. | High VRAM & storage cost. Slow. | Small models (<1B) or when you have massive compute resources. |
| **LoRA** | Freezes pre-trained weights, injects trainable rank decomposition matrices. | Low VRAM, fast training, small checkpoint size. | Slightly lower performance ceiling than full FT. | Most standard fine-tuning tasks on consumer GPUs. |
| **QLoRA** | LoRA + 4-bit quantization of base model (NF4). | Lowest VRAM usage. Can run 70B models on single GPU. | Slower training due to dequantization overhead. | Limited VRAM scenarios (e.g., Colab free tier). |

### 2. Important Hyperparameters
- **Learning Rate**: Controls step size during optimization. Too high = unstable; too low = slow convergence. For LoRA, usually higher (2e-4) than full FT.
- **Batch Size**: Number of samples processed before updating weights. Affects gradient stability and VRAM usage.
- **LoRA Rank (r)**: Dimension of the low-rank matrices. Higher r = more trainable params = more capacity, but higher VRAM.
- **LoRA Alpha**: Scaling factor for LoRA weights. Usually set to 2*r. Controls the strength of the adapter.
- **Epochs**: Number of full passes through the dataset. Too many = overfitting; too few = underfitting.
- **Warmup Steps**: Gradually increases LR from 0 to target value to stabilize early training.

### 3. Preventing Overfitting
1.  **Early Stopping**: Stop training when validation loss stops improving.
2.  **Dropout**: Randomly zero out neurons (LoRA dropout) to prevent reliance on specific features.
3.  **Regularization (Weight Decay)**: Penalize large weights to keep the model simple.



## 3. Training & Comparison
We run `scripts/train_comparison.py` to train multiple LoRA configurations and compare them.
**Note**: This step may take significant time. If models are already trained, it will skip training and generate reports.


In [4]:
!python ../scripts/train_comparison.py


INFO     2025-12-01 12:40:21 - setup - INFO - Logging initialized. Log file:    
         logs/project_20251201_124021.log                                       
╭──────────────────────────────────────────────────────────────────────────────╮
│ Running LoRA Experiments (Subprocess)                                        │
╰──────────────────────────────────────────────────────────────────────────────╯
INFO     2025-12-01 12:40:21 - __main__ - INFO - Checking for results in:       
         /home/thanhnx/qwen/finetune-qwen2.5-1.5b-ai-medical-chatbot/outputs/lor
         a_r16_alpha32                                                          


INFO     2025-12-01 12:40:24 - setup - INFO - Logging initialized. Log file:    
         logs/project_20251201_124024.log                                       
INFO     2025-12-01 12:40:24 - setup - INFO - Logging initialized. Log file:    
         logs/project_20251201_124024.log                                       
INFO     2025-12-01 12:40:24 - setup - INFO - Logging initialized. Log file:    
         logs/project_20251201_124024.log                                       
╭──────────────────────────────────────────────────────────────────────────────╮
│ LoRA Fine-tuning (lora_r16_alpha32)                                          │
╰──────────────────────────────────────────────────────────────────────────────╯
INFO     2025-12-01 12:40:24 - __main__ - INFO - Overriding LoRA config: {'r':  
         16, 'lora_alpha': 32}                                                  
INFO     2025-12-01 12:40:24 - __main__ - INFO - Configuration loaded.          


wandb: Currently logged in as: xt2201-vht (xt2201-vht-hust) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


wandb: ⢿ setting up run yt0wrox3 (0.0s)


...


m

m
wandb: ⣽ uploading output.log 40.4KB/40.4KB (1.2s)
wandb: ⣽ uploading config.yaml 15.7KB/15.7KB (0.8s)



m

m
wandb: ⣾ uploading history steps 53-53, summary, console lines 264-280 (0.3s)



m
wandb: ⣷ uploading history steps 53-53, summary, console lines 264-280 (0.3s)



m
wandb: 
wandb: Run history:
wandb:             eval/entropy █▃▃▂▂▂▂▂▂▂▂▁▁▁▁▂
wandb:                eval/loss █▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: eval/mean_token_accuracy ▁▇██████████████
wandb:          eval/num_tokens ▁▂▂▂▃▃▄▄▅▆▆▆▇▇██
wandb:             eval/runtime ▆▃▂▃▁▂▅▆▇▂▃▂▂█▇▇
wandb:  eval/samples_per_second ▃▆▇▆█▇▄▃▂▇▆▇▇▁▂▂
wandb:    eval/steps_per_second ▃▆▆▅█▇▄▃▂▆▆▇▇▁▂▂
wandb:            train/entropy █▇▅▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▂▂▂▁▂▁▂▂▁▂▂▁▁▁▂▁▁▁▁
wandb:              train/epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
wandb:        train/global_step ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
wandb:                       +5 ...
wandb: 
wandb: Run summary:
wandb:             eval/entropy 1.31815
wandb:                eval/loss 1.36981
wandb: eval/mean_token_accuracy 0.69934
wandb:          eval/num_tokens 2290137.0
wandb:             eval/runtime 17.2466
wandb:  eval/samples_per_second 28.991
wandb:    eval/steps_per_second 3.653
wandb:               total_flos 3.0033432275337216e+16
wandb:      

INFO     2025-12-01 13:51:12 - __main__ - INFO - Experiment lora_r32_alpha64    
         completed successfully.                                                
Traceback (most recent call last):
  File "/home/thanhnx/qwen/finetune-qwen2.5-1.5b-ai-medical-chatbot/notebooks/../scripts/train_comparison.py", line 246, in <module>
    run_experiments()
    ~~~~~~~~~~~~~~~^^
  File "/home/thanhnx/qwen/finetune-qwen2.5-1.5b-ai-medical-chatbot/notebooks/../scripts/train_comparison.py", line 243, in run_experiments
    generate_report(results)
    ~~~~~~~~~~~~~~~^^^^^^^^^
  File "/home/thanhnx/qwen/finetune-qwen2.5-1.5b-ai-medical-chatbot/notebooks/../scripts/train_comparison.py", line 49, in generate_report
    with open(output_file, "w") as f:
         ~~~~^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'outputs/experiment_report.md'


## Task 4: Analysis & Conclusion

### 1. Is the model overfitting?
**Answer**: Based on the loss curves (see `outputs/experiment_report.md` or WandB), if the validation loss starts increasing while training loss decreases, the model is overfitting. In our experiments, we used **Early Stopping** and **Dropout (0.2)** to mitigate this. The reduced epoch count (3) also helps.

### 2. Improvements for Performance
- **Data Quality**: Clean and filter the medical dataset further to remove noise.
- **Hyperparameter Tuning**: Run a sweep for Learning Rate and LoRA Rank.
- **Few-shot Prompting**: Include examples in the prompt context during inference.
- **RAG (Retrieval Augmented Generation)**: Connect the model to a medical knowledge base for up-to-date info.

### 3. Production Considerations
- **Latency**: LoRA adapters add minimal latency, but 4-bit quantization might slow down inference slightly.
- **Safety**: Medical advice requires strict guardrails. The model should refuse to answer dangerous questions or include a disclaimer.
- **Monitoring**: Track user feedback and drift in query distribution.



## 4. Results Report
Here is the summary of our experiments.


In [5]:
report_path = "../outputs/experiment_report.md"
if os.path.exists(report_path):
    with open(report_path, "r") as f:
        display(Markdown(f.read()))
else:
    print("Report not found. Please run training first.")


Report not found. Please run training first.


### Loss Curves
Visualizing the training stability.


In [6]:
# Display loss curves for the experiments
experiments = ["lora_r16_alpha32_v3", "lora_r16_alpha32_lowlr", "lora_r32_alpha64_v2"]
for exp in experiments:
    img_path = f"../outputs/{exp}/loss_curve.png"
    if os.path.exists(img_path):
        display(Markdown(f"#### {exp}"))
        display(Image(filename=img_path))


## 5. Comparative Inference
We compare the generation quality of different LoRA configurations on the same set of questions.


In [7]:
# Load Base Model Once
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

print(f"Loading base model: {model_name}...")
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True
)
print("Base model loaded.")

def generate_response(model, question, max_new_tokens=256):
    prompt = f"<|im_start|>system\nYou are a helpful medical assistant.<|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            do_sample=True
        )
    
    response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    return response.strip()

# Test Questions for Comparison
comparison_questions = [
    "What are the symptoms of diabetes?",
    "How to treat a common cold?"
]

# Iterate and Compare
experiments = ["lora_r16_alpha32_v3", "lora_r16_alpha32_lowlr", "lora_r32_alpha64_v2"]
results = {}

# Initialize PeftModel with the first adapter to get the class wrapper
first_exp = experiments[0]
first_adapter_path = f"../outputs/{first_exp}/final_checkpoint"

if os.path.exists(first_adapter_path):
    print(f"Initializing PeftModel with {first_exp}...")
    model = PeftModel.from_pretrained(base_model, first_adapter_path, adapter_name=first_exp)
else:
    print(f"Warning: {first_adapter_path} not found. Skipping {first_exp}.")
    model = None

if model:
    for exp in experiments:
        adapter_path = f"../outputs/{exp}/final_checkpoint"
        if not os.path.exists(adapter_path):
            continue
            
        if exp != first_exp:
            print(f"Loading adapter: {exp}...")
            model.load_adapter(adapter_path, adapter_name=exp)
            
        print(f"Switching to adapter: {exp}")
        model.set_adapter(exp)
        
        print(f"Generating responses for {exp}...")
        exp_results = []
        for q in comparison_questions:
            ans = generate_response(model, q)
            exp_results.append(ans)
        results[exp] = exp_results

    # Print Comparison
    for i, q in enumerate(comparison_questions):
        print(f"\n=== Question: {q} ===")
        for exp, answers in results.items():
            print(f"\n[{exp}]:\n{answers[i]}")
            print("-" * 30)



Loading base model: Qwen/Qwen2.5-1.5B-Instruct...


Base model loaded.


## 6. Inference with Best Model
We load the best performing checkpoint (dynamically detected) and test it on diverse medical queries.


In [8]:
# Dynamic Best Model Detection
import json
import glob

def get_best_model_path(outputs_dir="../outputs"):
    best_loss = float("inf")
    best_exp = None
    
    # Iterate over all experiment directories
    exp_dirs = [d for d in os.listdir(outputs_dir) if os.path.isdir(os.path.join(outputs_dir, d))]
    
    for exp in exp_dirs:
        trainer_state_path = os.path.join(outputs_dir, exp, "final_checkpoint", "trainer_state.json")
        if not os.path.exists(trainer_state_path):
             # Fallback to checking the directory itself if final_checkpoint doesn't have it (rare)
             trainer_state_path = os.path.join(outputs_dir, exp, "trainer_state.json")
        
        if os.path.exists(trainer_state_path):
            try:
                with open(trainer_state_path, "r") as f:
                    data = json.load(f)
                    # Check for best_metric (usually eval_loss)
                    loss = data.get("best_metric")
                    if loss is None and "log_history" in data:
                        # Try to find min eval_loss in history if best_metric not set
                        eval_losses = [x["eval_loss"] for x in data["log_history"] if "eval_loss" in x]
                        if eval_losses:
                            loss = min(eval_losses)
                            
                    if loss is not None and loss < best_loss:
                        best_loss = loss
                        best_exp = exp
            except Exception as e:
                print(f"Error reading {trainer_state_path}: {e}")
                
    if best_exp:
        return os.path.join(outputs_dir, best_exp, "final_checkpoint"), best_exp
    return None, None

adapter_path, best_exp_name = get_best_model_path()

if adapter_path:
    print(f"Automatically selected best model: {best_exp_name}")
    print(f"Adapter path: {adapter_path}")
    
    # Load the best adapter
    # Note: 'model' is already a PeftModel from previous cell. We can just load this new one.
    if 'model' in locals() and model is not None:
        print(f"Loading best adapter {best_exp_name} into existing model...")
        try:
            model.load_adapter(adapter_path, adapter_name="best_model")
            model.set_adapter("best_model")
        except Exception as e:
             print(f"Adapter might already exist or error: {e}. Trying to set it if exists.")
             if best_exp_name in model.peft_config:
                 model.set_adapter(best_exp_name)
             else:
                 # Fallback: Re-init if something is wrong
                 print("Re-initializing model for best checkpoint...")
                 model = PeftModel.from_pretrained(base_model, adapter_path, adapter_name="best_model")
    else:
        model = PeftModel.from_pretrained(base_model, adapter_path)
        
    model.eval()
    print("Best model loaded successfully!")

else:
    print("Could not find any trained models.")



Automatically selected best model: lora_r32_alpha64
Adapter path: ../outputs/lora_r32_alpha64/final_checkpoint


Best model loaded successfully!


In [9]:
def generate_response(question, max_new_tokens=256):
    prompt = f"<|im_start|>system\nYou are a helpful medical assistant.<|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            do_sample=True
        )
    
    response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    return response.strip()

# Diverse Use Cases
test_cases = [
    "What are the early warning signs of diabetes?",
    "I have a severe headache and sensitivity to light. What could it be?",
    "Explain the difference between viral and bacterial infections.",
    "What is the recommended treatment for a sprained ankle?",
    "Can you explain how vaccines work to a 5-year-old?"
]

print("--- Inference Results ---")
for i, case in enumerate(test_cases, 1):
    print(f"\nCase {i}: {case}")
    response = generate_response(case)
    print(f"Response: {response}")
    print("-" * 50)


--- Inference Results ---

Case 1: What are the early warning signs of diabetes?


Response: Hi. If you have any symptoms which I am not able to see in your attached report (attachment removed to protect patient identity), then you can send me those, so that I will be able to help you better. Diabetes is characterized by high blood sugar levels over a long period of time. It usually develops slowly and does not cause many symptoms until it gets severe. The following may indicate the presence of diabetes or pre-diabetes: 1) frequent urination (increased frequency of need to urinate). 2) thirst for drinking more water. 3) unexplained weight loss (loss of body mass). 4) blurry vision. 5) fatigue, weakness. 6) numbness and tingling sensations around the extremities (feet, hands, legs). 7) recurrent infections. 8) increased appetite. You should take this test after at least six months if you are already on treatment. For further information consult an endocrinologist online -->
--------------------------------------------------

Case 2: I have a severe headache and sensit

Response: Hi. I understand your concern. I would suggest you take tablet Paracetamol 500 mg thrice daily for five days, tablet Diclofenac 25 mg twice daily for seven days and tablet Sumatriptan 10 mg once at night. You can also try combination of tablet Aceclofenac + Paracetamol 500/325 mg twice daily for one week. Also, please avoid bright lights in the room. Try using an eye shield over eyes while watching TV or computer screen. I hope this helps. Please revert back with more information about your condition. Hope this helps. For further queries consult a neurologist online -->
--------------------------------------------------

Case 3: Explain the difference between viral and bacterial infections.


Response: Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult an internal medicine physician online -->
--------------------------------------------------

Case 4: What is the recommended treatment for a sprained ankle?


Response: Hello. I have gone through your query and understand your concerns. It appears to be a grade 1 ligament injury with soft tissue damage. You can apply ice packs, but not too much pressure on it. Do some gentle stretching of the calf muscle. Take anti-inflammatory medication like Ibuprofen. If there is any swelling you should take an injection called Dexamethasone which will help reduce inflammation and pain. Also, you need to use crutches in order to avoid further aggravation of the injury. Once the initial symptoms subside, you can start doing exercises that will help improve the strength of the muscles around the ankle joint and also prevent future injuries. You can see a physiotherapist or orthopedician who would advise you about specific exercises. In case you require more information, please do not hesitate to contact me.
--------------------------------------------------

Case 5: Can you explain how vaccines work to a 5-year-old?


Response: Hi. The vaccine is like a message that tells your immune system when to make antibodies (specialized cells) against the virus. So, after getting vaccinated, if it comes in contact with that virus, then your body will be ready and can fight against it without causing any symptoms. But if there was no vaccination, then this time may cause disease or illness. I hope it has helped you. If you have any further queries, please feel free to ask me.
--------------------------------------------------
